## Categorical Feature Encoding Challenge
- 해당 조건에서 target(고양이)가 있는지 없는지 예측하는 문제
- score: AUC
- [참조한 커널](https://www.kaggle.com/deepshekhar/categorical-feature-encoding-challenge)

In [1]:
import numpy as np
import pandas as pd

In [29]:
%%time
train = pd.read_csv("../input/cat_chal/train.csv")
test = pd.read_csv("../input/cat_chal/test.csv")

print(train.shape)
print(test.shape)

(300000, 25)
(200000, 24)
Wall time: 1.9 s


In [13]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [23]:
for col in train.columns:
    if "nom" in col:
        print(train[col].head(3))
        print("-" * 20)

0    Green
1    Green
2     Blue
Name: nom_0, dtype: object
--------------------
0     Triangle
1    Trapezoid
2    Trapezoid
Name: nom_1, dtype: object
--------------------
0      Snake
1    Hamster
2       Lion
Name: nom_2, dtype: object
--------------------
0    Finland
1     Russia
2     Russia
Name: nom_3, dtype: object
--------------------
0     Bassoon
1       Piano
2    Theremin
Name: nom_4, dtype: object
--------------------
0    50f116bcf
1    b3b4d25d0
2    3263bdce5
Name: nom_5, dtype: object
--------------------
0    3ac1b8814
1    fbcb50fc1
2    0922e3cb8
Name: nom_6, dtype: object
--------------------
0    68f6ad3e9
1    3b6dd5612
2    a6a36f527
Name: nom_7, dtype: object
--------------------
0    c389000ab
1    4cd920251
2    de9c9f684
Name: nom_8, dtype: object
--------------------
0    2f4cb3d51
1    f83c56c21
2    ae6800dd0
Name: nom_9, dtype: object
--------------------


### 일단 lightGBM으로 때려 박아봄

In [6]:
import lightgbm as lgb
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
import gc, os, pickle

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [7]:
for col in train.columns:
    print(col, "dtype:", train[col].dtype)

id dtype: int64
bin_0 dtype: int64
bin_1 dtype: int64
bin_2 dtype: int64
bin_3 dtype: object
bin_4 dtype: object
nom_0 dtype: object
nom_1 dtype: object
nom_2 dtype: object
nom_3 dtype: object
nom_4 dtype: object
nom_5 dtype: object
nom_6 dtype: object
nom_7 dtype: object
nom_8 dtype: object
nom_9 dtype: object
ord_0 dtype: int64
ord_1 dtype: object
ord_2 dtype: object
ord_3 dtype: object
ord_4 dtype: object
ord_5 dtype: object
day dtype: int64
month dtype: int64
target dtype: int64


In [30]:
%%time
# object 형식 data type이 많음 -> Label Encoding
for col in train.columns:
    if train[col].dtype == 'object':
        le = LabelEncoder()
        # ieee-cis에선 아래에서 train, test를 모두 fit 해줬는데, train으로만 해주는 게 맞는 듯.
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))
        
#Category Encoders (github) -> train에 안 나왔던게 test에도 나왔기 때문

ValueError: y contains previously unseen labels: ['1f0a80e1d', '2be51c868', 'a9bf3dc47', 'ec337ce4c']

In [25]:
from sklearn.model_selection import KFold

params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }

In [26]:
%%time
X = train.drop(['target'], axis=1)
y = train['target']

X_test = test

del train, test
gc.collect()

Wall time: 153 ms


11

In [27]:
print(X.shape)
print(X_test.shape)

(300000, 24)
(200000, 24)


In [28]:
%%time

NFOLDS = 5
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields nom_8, nom_9